In [1]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [2]:
def weight_variable(shape):
        initializer = tf.glorot_normal_initializer()
        init_val = initializer(shape)
        return tf.Variable(init_val)
    

def bias_variable(shape):
    init_val = tf.constant(0.01,shape=shape)
    return tf.Variable(init_val)
    

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        

def nn_block(x,x_channels,y_channels,name,size_of_filters=3,act=tf.nn.relu):
    with tf.name_scope(name):
        if name=='cnn':
            with tf.name_scope('weights'):
                weight = weight_variable([size_of_filters,size_of_filters,x_channels,y_channels])
                #variable_summaries(weight)
            with tf.name_scope('biases'):
                bias = bias_variable([y_channels])
                #variable_summaries(bias)
            with tf.name_scope('conv_layer'):
                pre_active = tf.add(tf.nn.conv2d(x,filter=weight,strides=1,padding='SAME'),bias)
                tf.summary.histogram('pre_activations',pre_active)
                                  
        elif name=='dense':
            with tf.name_scope('weights'):
                weight = weight_variable([x_channels,y_channels])
                #variable_summaries(weight)
            with tf.name_scope('biases'):
                bias = bias_variable([y_channels])
                #variable_summaries(bias)
            with tf.name_scope('dense_layer'):
                pre_active = tf.add(tf.matmul(x,weight),bias)
                tf.summary.histogram('pre_activations',pre_active)
                                      
        active = act(pre_active,name='activation')  
        tf.summary.histogram('activation',active)
        return active           

In [3]:
def train(lr=0.0005,epochs=10,keep_prob=0.4):
    mnist = input_data.read_data_sets('/tmp/data/')
    sess = tf.InteractiveSession()

    show_images = 1
    log_dir = '/home/sid/Codes/mnist_logs'
    
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32,[None,784],name='x_input')
        y = tf.placeholder(tf.int64,[None],name='y_input')
        
    #with tf.name_scope('input_reshape'):
    #    image_input = tf.reshape(x,[-1,28,28,1])
    #tf.summary.image('input',image_input,show_images) #10 images are shown

    conv_1 = nn_block(tf.reshape(x,[-1,28,28,1]),1,64,'cnn')
    #with tf.name_scope('pool_1'):
    conv_1 = tf.nn.max_pool(conv_1,2,2,'VALID')
        #conv_1_reshape = tf.reshape(conv_1,[-1,56,56,4])
    #tf.summary.image('pooling',conv_1_reshape,show_images)
    
    conv_2 = nn_block(conv_1,64,64,'cnn')
    #with tf.name_scope('pool_2'):
    conv_2 = tf.nn.max_pool(conv_2,2,2,'VALID')
        #conv_2_reshape = tf.reshape(conv_2,[-1,28,28,4])
    #tf.summary.image('pooling',conv_2_reshape,show_images)
        
    conv_2 = tf.reshape(conv_2,[-1,7*7*64])
    dense_1 = nn_block(conv_2,7*7*64,256,'dense')
    dense_1 = tf.nn.dropout(dense_1,rate=(1-keep_prob))
    output = nn_block(dense_1,256,10,'dense',act=tf.identity)
    
    #with tf.name_scope('cross_entropy'):
    #    with tf.name_scope('total'):
    cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=output)
    tf.summary.scalar('cross_entropy',cross_entropy)
    
    
    #with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(cross_entropy)
        
    #with tf.name_scope('accuracy'):
    #    with tf.name_scope('correct_prediction'):
    correct_prediction = tf.equal(tf.argmax(output,1),y)
    #    with tf.name_scope('accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('accuracy',accuracy)
    
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_dir+'/train',sess.graph)
    test_writer = tf.summary.FileWriter(log_dir+'/test')
    tf.global_variables_initializer().run()
    
    def feed_dict(train):
        if train:
            xs,ys = mnist.train.next_batch(5)
        else:
            xs,ys = mnist.test.images, mnist.test.labels
            
        print('************************************')
        print(xs.shape,xs.dtype,ys.shape,ys.dtype)
        return {x:xs,y:ys}
    
    for epoch in range(1,epochs+1):
        if epoch%10==0:
            summary,acc = sess.run([merged,accuracy],feed_dict=feed_dict(False))
            test_writer.add_summary(summary,epoch)
            print('Accuracy at step %s: %s'%(epoch,acc))
        else:
            summary,_ = sess.run([merged,train_step],feed_dict=feed_dict(True))
            train_writer.add_summary(summary,epoch)
    
    train_writer.close()
    test_writer.close()
        
with tf.Graph().as_default():        
    train()

W0719 12:19:18.056686 139797570033472 deprecation.py:323] From <ipython-input-3-e0712c714e79>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0719 12:19:18.058565 139797570033472 deprecation.py:323] From /home/sid/tensorflow/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0719 12:19:18.060748 139797570033472 deprecation.py:323] From /home/sid/tensorflow/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future 

Extracting /tmp/data/train-images-idx3-ubyte.gz


W0719 12:19:18.500131 139797570033472 deprecation.py:323] From /home/sid/tensorflow/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0719 12:19:18.577004 139797570033472 deprecation.py:323] From /home/sid/tensorflow/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


W0719 12:19:19.119964 139797570033472 deprecation.py:506] From /home/sid/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0719 12:19:19.272203 139797570033472 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0719 12:19:19.339356 139797570033472 deprecation.py:323] From /home/sid/tensorflow/lib/python3.6/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


************************************
(5, 784) float32 (5,) uint8
************************************
(5, 784) float32 (5,) uint8
************************************
(5, 784) float32 (5,) uint8
************************************
(5, 784) float32 (5,) uint8
************************************
(5, 784) float32 (5,) uint8
************************************
(5, 784) float32 (5,) uint8
************************************
(5, 784) float32 (5,) uint8
************************************
(5, 784) float32 (5,) uint8
************************************
(5, 784) float32 (5,) uint8
************************************
(10000, 784) float32 (10000,) uint8


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[10000,64,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node cnn/conv_layer/Conv2D (defined at <ipython-input-2-27e7727b2645>:34) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[10000,64,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node cnn/conv_layer/Conv2D (defined at <ipython-input-2-27e7727b2645>:34) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[dense_1/activation/_41]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOption